In [ ]:
# We need older numpy version, otherwise an error occurs (newer numpy has no member "int", "float",...)
# TODO testing lower version of openmim - 0.3.2 is on dell 7400 (was 0.3.5) - did not work
%pip install opencv-python numpy==1.23.5 openmim torchvision

In [ ]:
from pprint import pprint
import numpy as np
import cv2
import csv
import json
import os
import sys
import shutil
import pickle
import traceback

# Classes are defined in process_dataset/common.py
from process_dataset.common import classes, classes_dict, datasets_path, dataset_pickle_filepath

### Additional paths

In [ ]:
import paths

with open("./paths.py") as f:
    print(f.read())

# proj_path = os.getcwd()

# drive_dirpath = os.path.join(proj_path, "drive")
# drive_traffic_cams_path = os.path.join(drive_dirpath, "traffic_cams")

# mmdetection_path = os.path.join(proj_path, "..", "mmdetection")

# working_dirpath = os.path.join(proj_path, "working_dir")

# process_dataset_dirpath = os.path.join(proj_path, "process_dataset")

# !mkdir -p $working_dirpath

# ! Do this in a separate terminal:
# !mkdir -p $drive_dirpath
# !rclone mount remote: $drive_dirpath

# Don't forget to unmount when you're finished

# TODO assert everything is in the right place

# Installing mmdetection

In [ ]:
# We install older mmcv version, otherwise error occurs (mmdet says newest possible is 1.7.0)
!mim install mmcv-full==1.7.0
%cd $paths.mmdetection_path/..
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
%pip install -e .
%cd $paths.proj_path

In [ ]:
#@title Verify installation
import mmdet
print(mmdet.__version__)

In [ ]:
from mmcv import collect_env
try:
    collect_env() # TODO throws error
except Exception as e:
    print("Could not run 'collect_env()'. Exception:")
    traceback.print_exc()


# Download YOLOX-s config and checkpoint (pre-trained)

In [ ]:
# TODO don't download it here. Leave it to the user
# Memory requirements for YOLOX models: 
# YOLOX-s	7.6 GB
# YOLOX-l	19.9 GB
# YOLOX-x	28.1 GB

# url = "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/" + model_checkpoint_filename
# url = "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_s_8x8_300e_coco/" + paths.model_checkpoint_filename
# !wget -c $url -O $paths.model_checkpoint_filepath

In [ ]:
# https://colab.research.google.com/github/ZwwWayne/mmdetection/blob/update-colab/demo/MMDet_Tutorial.ipynb#scrollTo=hamZrlnH-YDD
from mmdet.apis import set_random_seed
from mmcv import Config

cfg = Config.fromfile(paths.model_config_filepath)

# print(f'Initial config:\n{cfg.pretty_text}')

if paths.last_checkpoint_filepath:
    cfg.resume_from = paths.last_checkpoint_filepath
else:
    cfg.load_from = paths.model_checkpoint_filepath

cfg.work_dir = paths.working_dirpath

cfg.data_root = datasets_path

img_prefix = datasets_path

# dataset_filepath = os.path.join(dataset_mio_tcd_path, "gt.pickle")
dataset_filepath = os.path.join(datasets_path, "dataset.pickle")
train_filepath = os.path.join(datasets_path, "train.pickle")
val_filepath = os.path.join(datasets_path, "val.pickle")
test_filepath = os.path.join(datasets_path, "test.pickle")

cfg.data.train.dataset.type = "CustomDataset"
cfg.data.train.dataset.ann_file = train_filepath
cfg.data.train.dataset.img_prefix = img_prefix
cfg.data.val.type = "CustomDataset"
cfg.data.val.ann_file = val_filepath
cfg.data.val.img_prefix = img_prefix
cfg.data.test.type = "CustomDataset"
cfg.data.test.ann_file = test_filepath
cfg.data.test.img_prefix = img_prefix

cfg.data.train.dataset = {
    "type": 'ClassBalancedDataset',
    # "oversample_thr": 1e-3,
    # "oversample_thr": 0.01,
    "oversample_thr": 0.1,
    "dataset": cfg.data.train.dataset
}

"""
img_norm_cfg = {
    # "mean": [103.530, 116.280, 123.675], # Taken from yolof
    "mean": [114.0, 114.0, 114.0], 
    "std": [1.0, 1.0, 1.0], 
    "to_rgb": False
}
"""

cfg.train_pipeline = [
    # dict(type='Mosaic', # Not really useful here
    #      img_scale=cfg.img_scale, 
    #      pad_val=114.0),
    dict(type='RandomAffine',
        # min_bbox_size=8, # No need. Done in FilterAnnotations
        # border=(-cfg.img_scale[0] // 2, -cfg.img_scale[1] // 2), # This was the problem. No idea why I added it. Shouldn't exist
        scaling_ratio_range=(0.1, 2),
        max_rotate_degree=15,
        max_shear_degree=10),
    # dict(type='MixUp', # Not really useful here
    #     img_scale=cfg.img_scale,
    #     ratio_range=(0.8, 1.6),
    #     pad_val=114.0),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip',
         flip_ratio=0.5,
         direction="horizontal"), # (horizontal is implicit)
    # According to the official implementation, multi-scale
    # training is not considered here but in the
    # 'mmdet/models/detectors/yolox.py'.
    # TODO RandomCrop?
    dict(type='Resize',
         img_scale=cfg.img_scale,
         keep_ratio=True),
    dict(type='Pad',
        pad_to_square=True,
        # If the image is three-channel, the pad value needs
        # to be set separately for each channel.
        pad_val=dict(img=(114.0, 114.0, 114.0))),
    dict(type='FilterAnnotations',
        # min_gt_bbox_wh=(8, 8), # TODO Should be okay but I can try increasing
        min_gt_bbox_wh=(16, 16),
        keep_empty=False),
    dict(type="PhotoMetricDistortion"),
    # Is this OK? Heard that YOLOX does not need normalization or something...
    # Nope, it definitely does not. I tried one epoch with and one without, and
    # the results (loss plots) were actually the same
    # dict(type='Normalize', **img_norm_cfg), # img_norm_cfg taken from yolof
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
cfg.train_dataset.pipeline = cfg.train_pipeline
cfg.data.train.pipeline = cfg.train_pipeline

cfg.gpu_ids = [0]
cfg.device = "cuda"

cfg.seed = 0
set_random_seed(0, deterministic=False)

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
# cfg.optimizer.lr = 0.02 # This instead of 0.02 / 8 - nope, that's too much
cfg.optimizer.lr /= 8 # 0.00125
# cfg.optimizer.lr = 0.001
# cfg.optimizer.lr = 0.0001 # bak4

# Batch size (default 8)
# cfg.data.samples_per_gpu = 32 # Let's try 32 - nope, cuda out of memory on 7400
# cfg.data.samples_per_gpu = 16 # nope, cuda out of memory on 7400
# cfg.data.samples_per_gpu = 12 # This seems to work (11111 of 11178MiB used) - not on P52
cfg.data.samples_per_gpu = 11 # Works on P52
# cfg.data.samples_per_gpu = 10

# Workers per gpu (default 4)
# Tested 8, 12 and 16 on P52 and higher numbers actually made the training (ETA) longer
# With 12, ETA was about 10% longer than at default. Using 2, speed is slightly improved (~2%)
# cfg.data.workers_per_gpu = 8 # Doesn't seem to do much

# Orig yolox-s config says: USER SHOULD NOT CHANGE ITS VALUES
# ! But that, hopefully, means that user should not change LR, but can change
# the auto_scale_lr setting...
# cfg.auto_scale_lr = {}
cfg.auto_scale_lr = {
    "enable": True, 
    "base_batch_size": cfg.data.samples_per_gpu
    }

# Reduce the lr warmup to one epoch (instead of 5)
# cfg.lr_config.warmup_iters = 1
# cfg.lr_config.warmup = None

# cfg.log_config.interval = 100
cfg.log_config.interval = 50

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'

# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

cfg.max_epochs = 300
cfg.runner = dict(type='EpochBasedRunner', max_epochs=cfg.max_epochs)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

# TODO validation sometimes
# cfg.workflow = [('train', 1), ('val', 1)]

cfg.pop("train_dataset")
cfg.pop("train_pipeline")
cfg.pop("test_pipeline")

# Set number of classes
cfg.model.bbox_head.num_classes = len(classes)

# Tried this but it didn't help with the mmdetection's concat problem
# cfg.data.train.dataset["filter_empty_gt"] = True

In [ ]:
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
# https://colab.research.google.com/github/ZwwWayne/mmdetection/blob/update-colab/demo/MMDet_Tutorial.ipynb#scrollTo=hamZrlnH-YDD
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]
 
# Build the detector
# model = build_detector(cfg.model, test_cfg=cfg.data.test)
model = build_detector(cfg.model)

# Train
model.CLASSES = classes
train_detector(model, datasets, cfg)

In [ ]:
# from mmdet.apis import single_gpu_test
# from mmdet.datasets import build_dataloader, build_dataset
# from mmdet.utils import build_dp

# data_loader = build_dataloader(build_dataset(cfg.data.test), samples_per_gpu=64, workers_per_gpu=1)
# dp = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
# outputs = single_gpu_test(dp, data_loader, out_dir=paths.working_dirpath)